In [ ]:
from keras.models import Sequential
from keras.layers import Rescaling, Conv2D, Activation, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt


def make_cnn(input_shape, learning_rate):
    model = Sequential()

    model.add(Rescaling(1./255))
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(5))
    model.add(Activation('softmax'))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate), 
        loss=SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    return model


def generate_data(data_dir, augment_data, validation_split, batch_size, image_size, seed):
    if not augment_data:
        train_datagen = ImageDataGenerator(validation_split=validation_split)
        test_datagen = ImageDataGenerator(validation_split=validation_split)

        train_data = train_datagen.flow_from_directory(
            directory=data_dir,
            batch_size=batch_size,
            target_size=image_size,
            class_mode='sparse',
            subset='training',
            seed=seed
        )

        test_data = test_datagen.flow_from_directory(
            directory=data_dir,
            batch_size=batch_size,
            target_size=image_size,
            class_mode='sparse',
            subset='validation',
            seed=seed
        )
    else:
        train_datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest',
            validation_split=validation_split,
        )

        test_datagen = ImageDataGenerator(validation_split=validation_split)

        train_data = train_datagen.flow_from_directory(
            directory=data_dir,
            batch_size=batch_size,
            target_size=image_size,
            class_mode='sparse',
            subset='training',
            seed=seed
        )

        test_data = test_datagen.flow_from_directory(
            directory=data_dir,
            batch_size=batch_size,
            target_size=image_size,
            class_mode='sparse',
            subset='validation',
            seed=seed
        )

    return train_data, test_data


def fit_model(model, train_data, test_data, epochs):
    model.fit(train_data, epochs=epochs, verbose=0)

    print("Model Accuracy Stats:")
    print("Training Accuracy:", model.evaluate(train_data, verbose=0)[1])
    print("Testing Accuracy:", model.evaluate(test_data, verbose=0)[1])
    

def predict(model, class_map, images):
    for i, image in enumerate(images):
        prediction = model.predict(np.expand_dims(image, axis=0), verbose=0)

        print("Image", i+1)
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        max_index = np.argmax(prediction)
        print(f'Image {i+1} Prediction: {class_map[max_index]}')

In [ ]:
data_dir = 'pizza types'
image_dim = (256, 256, 3)
learning_rate= 0.001
augment_data = True
validation_split = 0.2
batch_size = 32
seed = 42
epochs = 20

model = make_cnn(image_dim, learning_rate)
train_data, test_data = generate_data(data_dir, augment_data, validation_split, 
                                      batch_size, image_dim[0:2], seed
)
print()
fit_model(model, train_data, test_data, epochs)
print()

In [ ]:
class_map = {v: k for k, v in train_data.class_indices.items()}

images = []
for filename in os.listdir('test pizzas'):
    file_path = os.path.join('test pizzas', filename)
    image = cv2.resize(cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB), image_dim[0:2])
    images.append(image)

predict(model, class_map, images)